In [24]:
%matplotlib inline
import os
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [25]:
data = pd.read_csv(
    os.path.join('train.csv.bz2')
)

usholidays = pd.read_csv(
    os.path.join('external_data.csv')
)
data_weather = pd.read_csv(
    os.path.join('external_data_copie.csv')
)
test = pd.read_csv(
    os.path.join('classeur.csv'), sep = ';'
)

In [26]:
X_merged2 = data.copy()

# Make sure that DateOfDeparture is of datetime format
X_merged2.loc[:, 'DateOfDeparture'] = pd.to_datetime(X_merged2['DateOfDeparture'])
# Encode the DateOfDeparture
X_merged2.loc[:, 'year'] = X_merged2['DateOfDeparture'].dt.year
X_merged2.loc[:, 'month'] = X_merged2['DateOfDeparture'].dt.month
X_merged2.loc[:, 'day'] = X_merged2['DateOfDeparture'].dt.day
X_merged2.loc[:, 'weekday'] = X_merged2['DateOfDeparture'].dt.weekday
X_merged2.loc[:, 'week'] = X_merged2['DateOfDeparture'].dt.week
X_merged2.loc[:, 'n_days'] = X_merged2['DateOfDeparture'].apply(
    lambda date: (date - pd.to_datetime("1970-01-01")).days )

X_merged2.drop('DateOfDeparture', axis=1, inplace=True)

X_merged = pd.merge(
        X_merged2, usholidays, how='left', on=['week','weekday'], sort=False )
X_merged['Holidays'].fillna(0, inplace=True)
X_merged['Name'].fillna("No", inplace=True)
X_merged[X_merged['Holidays']==1]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # Remove the CWD from sys.path while we load stuff.


,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,year,month,day,weekday,week,n_days,Holidays,Name
8,ATL,CLT,10.238095,9.175645,6.609877,2012,5,27,6,21,15487,1.0,MM_Day
12,IAH,LAX,13.629630,10.267310,9.178353,2012,6,23,5,25,15514,1.0,Summer
30,SFO,ATL,9.526316,9.765837,6.653056,2011,11,19,5,46,15297,1.0,Thanksgiving
34,LAS,JFK,17.621622,11.121931,11.686534,2012,8,31,4,35,15583,1.0,LB_Day
44,PHL,BOS,5.545455,7.785131,4.227615,2012,1,7,5,1,15346,1.0,Chritmas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8932,ATL,DEN,14.208333,9.834016,11.057082,2012,7,7,5,27,15528,1.0,Summer
8934,CLT,ATL,8.944444,9.963324,5.985003,2011,11,23,2,47,15301,1.0,Thanksgiving
8944,DEN,SFO,11.173913,10.174042,7.068832,2012,6,16,5,24,15507,1.0,Summer
8955,ORD,PHL,9.473684,10.125598,6.501012,2012,6,2,5,22,15493,1.0,Summer


In [28]:
X = data.copy()
X.loc[:, 'DateOfDeparture'] = pd.to_datetime(X.loc[:, 'DateOfDeparture'])
X_holidays = usholidays[['Date', 'Holiday']]
X_holidays.loc[:, 'Date'] = pd.to_datetime(X_holidays.loc[:, 'Date'])
X_holidays = X_holidays.rename(
        columns={'Date': 'DateOfDeparture'}
    )

X_merged = pd.merge(
        X, X_holidays, how='left', on=['DateOfDeparture'], sort=False
    )
X_merged['Holiday'].fillna("No", inplace=True)
X_merged[X_merged['Holiday'] != "No"]

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,Holiday
24,2012-11-12,EWR,BOS,11.000000,11.740518,8.711759,Veterans Day
68,2012-12-25,DEN,LAX,15.034483,10.391932,11.890821,Christmas Day
77,2013-01-21,DEN,ORD,9.631579,10.935530,7.158604,"Birthday of Martin Luther King, Jr."
119,2011-11-24,DEN,ORD,10.176471,8.285437,7.307832,Thanksgiving Day
161,2013-01-01,LAX,BOS,13.666667,9.814790,8.948571,New Year's Day
...,...,...,...,...,...,...,...
8788,2012-10-08,MSP,DEN,9.631579,11.003166,6.701663,Columbus Day
8802,2012-01-02,EWR,SFO,15.037037,11.269364,11.686552,New Year's Day
8819,2012-05-28,SFO,JFK,16.096774,12.293312,11.249755,Memorial Day
8843,2012-12-25,SFO,LAX,17.705882,11.048593,11.645078,Christmas Day


In [5]:
X_merged2 = X_merged.copy()

# Make sure that DateOfDeparture is of datetime format
X_merged2.loc[:, 'DateOfDeparture'] = pd.to_datetime(X_merged2['DateOfDeparture'])
# Encode the DateOfDeparture
X_merged2.loc[:, 'year'] = X_merged2['DateOfDeparture'].dt.year
X_merged2.loc[:, 'month'] = X_merged2['DateOfDeparture'].dt.month
X_merged2.loc[:, 'day'] = X_merged2['DateOfDeparture'].dt.day
X_merged2.loc[:, 'weekday'] = X_merged2['DateOfDeparture'].dt.weekday
X_merged2.loc[:, 'week'] = X_merged2['DateOfDeparture'].dt.week
X_merged2.loc[:, 'n_days'] = X_merged2['DateOfDeparture'].apply(
    lambda date: (date - pd.to_datetime("1970-01-01")).days )

#X_merged2.drop('DateOfDeparture', axis=1, inplace=True)
X_merged2

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # Remove the CWD from sys.path while we load stuff.


,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,Holiday,year,month,day,weekday,week,n_days
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647,No,2012,6,19,1,25,15510
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734,No,2012,9,10,0,37,15593
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883,No,2012,10,5,4,40,15618
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202,No,2011,10,9,6,40,15256
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159,No,2012,2,21,1,8,15391
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8897,2011-10-02,DTW,ATL,9.263158,10.427055,7.316967,No,2011,10,2,6,39,15249
8898,2012-09-25,DFW,ORD,12.772727,12.201552,10.641034,No,2012,9,25,1,39,15608
8899,2012-01-19,SFO,LAS,11.047619,10.508746,7.908705,No,2012,1,19,3,3,15358
8900,2013-02-03,ORD,PHL,6.076923,10.174042,4.030334,No,2013,2,3,6,5,15739


In [17]:
def _encode_dates(X):
    # Make sure that DateOfDeparture is of dtype datetime
    X = X.copy()  # modify a copy of X
    X.loc[:, "DateOfDeparture"] = pd.to_datetime(X['DateOfDeparture'])
    # Encode the date information from the DateOfDeparture columns
    X.loc[:, 'month'] = X['DateOfDeparture'].dt.month
    X.loc[:, 'day'] = X['DateOfDeparture'].dt.day
    X.loc[:, 'weekday'] = X['DateOfDeparture'].dt.weekday
    X.loc[:, 'week'] = X['DateOfDeparture'].dt.week
    X.loc[:, 'n_days'] = X['DateOfDeparture'].apply(
        lambda date: (date - pd.to_datetime("1970-01-01")).days
    )
    
    #ADD Holidays
    we = [4, 5, 6]
    ho_we = [51, 35, 21, 26, 46]
    ho = [1, 52, 27, 47]
    ho_0 = [36, 22]
    summer = range(22, 35)
    X.loc[:, 'holiday'] = np.where((X['week'].isin(ho)) |
                                    ((X['week'].isin(ho_we)) & (X['weekday'].isin(we))) |
                                    ((X['week'].isin(ho_0)) & (X['weekday'] == 0)) |
                                   ((X['week'].isin(summer)) & (X['weekday'] == 5))
                                   , 1, 0)
    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["DateOfDeparture"])  

X_merged2 = _encode_dates(data)
X_merged2

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':


,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,month,day,weekday,week,n_days,holiday
0,ORD,DFW,12.875000,12.331296,9.812647,6,19,1,25,15510,0
1,LAS,DEN,14.285714,10.775182,9.466734,9,10,0,37,15593,0
2,DEN,LAX,10.863636,11.083177,9.035883,10,5,4,40,15618,0
3,ATL,ORD,11.480000,11.169268,7.990202,10,9,6,40,15256,0
4,DEN,SFO,11.450000,11.269364,9.517159,2,21,1,8,15391,0
...,...,...,...,...,...,...,...,...,...,...,...
8897,DTW,ATL,9.263158,10.427055,7.316967,10,2,6,39,15249,0
8898,DFW,ORD,12.772727,12.201552,10.641034,9,25,1,39,15608,0
8899,SFO,LAS,11.047619,10.508746,7.908705,1,19,3,3,15358,0
8900,ORD,PHL,6.076923,10.174042,4.030334,2,3,6,5,15739,0


In [23]:
#Classic encoder

X_transform = X_merged2.copy()

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(X_merged2['Arrival'].unique())

X_merged2['Arrival'] = le.transform(X_merged2['Arrival'])
X_merged2['Departure'] = le.transform(X_merged2['Departure'])

#le.fit(X_merged['Holiday'].unique())
#X_merged2['Holiday'] = le.transform(X_merged2['Holiday'])
X_merged2

,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,month,day,weekday,week,n_days,holiday
0,15,4,12.875000,12.331296,9.812647,6,19,1,25,15510,0
1,9,3,14.285714,10.775182,9.466734,9,10,0,37,15593,0
2,3,10,10.863636,11.083177,9.035883,10,5,4,40,15618,0
3,0,15,11.480000,11.169268,7.990202,10,9,6,40,15256,0
4,3,19,11.450000,11.269364,9.517159,2,21,1,8,15391,0
...,...,...,...,...,...,...,...,...,...,...,...
8897,5,0,9.263158,10.427055,7.316967,10,2,6,39,15249,0
8898,4,15,12.772727,12.201552,10.641034,9,25,1,39,15608,0
8899,19,9,11.047619,10.508746,7.908705,1,19,3,3,15358,0
8900,15,16,6.076923,10.174042,4.030334,2,3,6,5,15739,0


In [20]:
#Categorical encoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

X_transform = X_merged2.copy()

categorical_encoder = OrdinalEncoder()
categorical_cols = [
    "Arrival", "Departure", "Holiday", "year", "month",
    "weekday"
]

preprocessor = make_column_transformer(
    (categorical_encoder, categorical_cols)
)

#X_transform = preprocessor.fit_transform(X_transform)

categorical_encoder.fit(X_transform[categorical_cols])
X_transform[categorical_cols] = categorical_encoder.transform(X_transform[categorical_cols])

KeyError: "['Holiday', 'year'] not in index"

In [21]:
#Numerical encoder
from sklearn.preprocessing import StandardScaler

numerical_scaler = StandardScaler()
numerical_cols = ["WeeksToDeparture", "std_wtd"]

numerical_scaler.fit(X_transform[numerical_cols])
X_transform[numerical_cols] = numerical_scaler.transform(X_transform[numerical_cols])

#X_transform["WeeksToDeparture"] = pd.qcut(X_transform["WeeksToDeparture"],labels=False, q=10)
#X_transform["std_wtd"] = pd.qcut(X_transform["std_wtd"], labels=False, q=10)

In [22]:
X_transform

,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,month,day,weekday,week,n_days,holiday
0,15,4,0.512573,12.331296,0.558487,6,19,1,25,15510,0
1,9,3,1.018752,10.775182,0.396807,9,10,0,37,15593,0
2,3,10,-0.209127,11.083177,0.195426,10,5,4,40,15618,0
3,0,15,0.012031,11.169268,-0.293328,10,9,6,40,15256,0
4,3,19,0.001267,11.269364,0.420375,2,21,1,8,15391,0
...,...,...,...,...,...,...,...,...,...,...,...
8897,5,0,-0.783396,10.427055,-0.608000,10,2,6,39,15249,0
8898,4,15,0.475876,12.201552,0.945677,9,25,1,39,15608,0
8899,19,9,-0.143112,10.508746,-0.331420,1,19,3,3,15358,0
8900,15,16,-1.926652,10.174042,-2.144180,2,3,6,5,15739,0


In [59]:
import random
random.seed(30)

from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

X = X_transform.copy()
y = X['log_PAX']
X.drop(['log_PAX'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y)

XGB =XGBRegressor()

#XGB.fit(X_train, y_train)

scores = cross_val_score(
    XGB, X, y, cv=5, scoring='neg_mean_squared_error'
)

rmse_scores = np.sqrt(-scores)

print(
    f"RMSE: {np.mean(rmse_scores):.4f} +/- {np.std(rmse_scores):.4f}"
)

[15:53:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:53:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:53:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:53:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:53:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 0.6162 +/- 0.0230
